<a href="https://colab.research.google.com/github/mitramir55/Tweet-Sentiment-Extraction/blob/main/TSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We're using MIT License

In [1]:
#!pip install transformers

In [1]:
import numpy as np
import pandas as pd
import os
import string
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import re

In [2]:
pd.set_option('display.max_colwidth', None)

### importing from Kaggle

In [3]:
! pip install -q kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mitramir5","key":"5c52071fc5b2c52439101cfead055fdf"}'}

In [5]:

# create a directory for it
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/


In [6]:

# change mode to full read and write access 
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c tweet-sentiment-extraction

  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 60.4MB/s]


In [8]:
! unzip /content/tweet-sentiment-extraction.zip

Archive:  /content/tweet-sentiment-extraction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


### Starting with the dataset

In [9]:
df = pd.read_csv('train.csv')

In [10]:
df_test = pd.read_csv('test.csv')

In [11]:
len_train = len(df)
len_test = len(df_test)

print(f'dataset length is {len_train}')
print(f'test dataset length is {len_test}')
print(f'ratio = {round(len_test / len_train, 2)}')

dataset length is 27481
test dataset length is 3534
ratio = 0.13


In [12]:
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,",negative


In [13]:
df.dropna(inplace=True, how='any')

In [14]:
df.isna().sum()

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [15]:
def jaccard_score(t1, t2):
    
    t1_words = set(t1.lower().split())
    t2_words = set(t2.lower().split())
    inter = t1_words.intersection(t2_words)
    return len(inter) / (len(t1_words) + len(t2_words) - len(inter))

In [17]:
df.loc[:, 'jaccard_score'] = df.apply(lambda x: jaccard_score(x['selected_text'], x['text']), axis=1)

In [ ]:
for text in df.loc[:, 'selected_text']:
    if re.findall('#', text): print(text)

I still smell of smoke  #kitchenfire
All the cool people I want to find for following today are #English, and I guess the English don`t tweet.
good news: finally finished my #EASactive workout that has been paused for 6 hours. bad news: my resistance band is torn
Simple my a#@
It looks like the office TV DOES get MLB Network... and it looks like MLBN will NOT be televising the DET/BAL game today  #wieters
there were attempts to somehow extend inner classes, which would be close to #closure, can`t find the ref ATM
#stackeoverflow
Goodmorning twitter, oh my gosh, i woke up soooo nice, lol ... oh hai thar twitterverse. Happy #mothersday everybody
#bash
Jaydiohead 'No Karma' ? http://twt.fm/91610 #musicmonday This is not the android I was looking for. But it`ll do.
I don`t think I`ve ever been up this early on a Sunday in a while. Meanwhile, the #bigweekend awaits!
WORS bike race at camp this weekend=a total of 1500 people. Going anywhere Sunday is not an option as they race right past the

In [18]:
df

,textID,text,selected_text,sentiment,jaccard_score
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,1.000000
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,0.200000
2,088c60f138,my boss is bullying me...,bullying me,negative,0.166667
3,9642c003ef,what interview! leave me alone,leave me alone,negative,0.600000
4,358bd9e861,"Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,",negative,0.214286
...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband lost his job and can`t afford it,d lost,negative,0.058824
27477,4f4c4fc327,"I`ve wondered about rake to. The client has made it clear .NET only, don`t force devs to learn a new lang #agile #ccnet",", don`t force",negative,0.083333
27478,f67aae2310,Yay good for both of you. Enjoy the break - you probably need it after such hectic weekend Take care hun xxxx,Yay good for both of you.,positive,0.272727
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,1.000000


In [34]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Histogram(x=df.loc[:, 'sentiment'], ), row=1, col=1)
fig.add_trace(go.Histogram(x=df_test.loc[:, 'sentiment']), row=1, col=2)

fig.show()

In [25]:
fig = px.histogram(df, x="sentiment", color="sentiment", title="Click on the legend items!")
fig.show()

In [23]:
import plotly.express as px

fig = px.histogram(df, x="jaccard_score", color="sentiment", 
                 marginal="box", title="Click on the legend items!", facet_col="sentiment")
fig.show()